# Moptibet
Moptibet gathers betting odd’s data from multiple sources, consolidate said data, and highlight betting opportunities, through both arbitrage due to inconsistent odds in the market and using predictive analytics to guess winners.

### Imports

In [1]:
import pandas as pd
import sqlalchemy as db
import sys
import configparser

In [2]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

### Web Scraping

#### Site Specific HTML Parsers

In [4]:
def draftkings_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    odds = [x.text for x in soup.findAll('span',{'class', input_dict['odds_class']})]
    # odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::3], outcomes[2::3], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [datetime.now()] * len(outcomes[::3])))

def fanduel_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [datetime.now()] * len(outcomes[::3])))

def bovada_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[2::7], odds[4::7], odds[3::7], [key] * len(outcomes[::3]), [datetime.now()] * len(outcomes[::3])))

#### Site Specific Scraper Mapping

In [5]:
prem_dir = {'draftkings':{'url':'https://sportsbook.draftkings.com/leagues/soccer/53591936',
                          'parser': draftkings_parse,
                          'args':{'outcomes_class':'sportsbook-outcome-cell__label',
                                  'odds_class':'sportsbook-odds american default-color'}},
            'fanduel':{'url':'https://sportsbook.fanduel.com/sports/navigation/730.1/7567.1',
                       'parser': fanduel_parse,
                       'args':{'outcomes_class':'name',
                               'odds_class':'selectionprice'}},
            'bovada':{'url':'https://www.bovada.lv/sports/soccer/england-premier-league',
                      'parser': bovada_parse,
                      'args':{'outcomes_class':'name',
                              'odds_class':'bet-price'}}
           }
           
output_cols = ['home_team','away_team', 'home_odds','tie_odds', 'away_odds', 'source', 'datetime']

#### Setting Scraper settings and appropriate driver

In [6]:
sys.platform

'linux'

In [7]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

if sys.platform == 'darwin':
    DRIVER_PATH = 'driver/mac/chromedriver'
elif sys.platform == 'linux' or sys.platform == 'linux2':
    DRIVER_PATH = 'driver/linux/chromedriver'
elif sys.platform == 'win32':
    DRIVER_PATH = 'driver/windows/chromedriver'

#### Iterate over sites, persisting outcomes, odds

In [8]:
class Web_Scraper:
    def __init__(self, map_dir, output_cols, scr_options=None):
        self.map_dir = map_dir
        self.output_cols = output_cols
        self.scr_options = scr_options
        
    def set_options(self):
        if self.scr_options is not None:
            options = Options()
            options.headless = self.scr_options['headless']
            options.add_argument(''.join('--window-size=',self.scr_options['width'],',',self.scr_options['height']))
        else:
            options = Options()
            options.headless = True
            options.add_argument('--window-size=1920,1200')
    
    def os_driver_path(self):
        if sys.platform == 'darwin':
            DRIVER_PATH = 'driver/mac/chromedriver'
        elif sys.platform == 'linux' or sys.platform == 'linux2':
            DRIVER_PATH = 'driver/linux/chromedriver'
        elif sys.platform == 'win32':
            DRIVER_PATH = 'driver/windows/chromedriver'
        return DRIVER_PATH
    
    def go(self):
        dn = []
        for key in self.map_dir:
            driver = webdriver.Chrome(options=self.set_options(), executable_path=self.os_driver_path())
            driver.get(self.map_dir[key]['url'])
            driver.implicitly_wait(3)
            
            data = pd.DataFrame(prem_dir[key]['parser'](prem_dir[key]['args'],driver, key),columns=self.output_cols)
            dn.append(data)
            driver.quit()
        return pd.concat(dn, axis=0)
   
    ##########
    #
    # N E X T
    #
    ##########

    def update_db(self):
        pass

        home_team       away_team home_odds tie_odds away_odds      source  \
0          Fulham         Arsenal      +450     +340      -177  draftkings   
1  Crystal Palace     Southampton      +185     +215      +160  draftkings   
2        West Ham       Newcastle      +120     +225      +245  draftkings   
3       Liverpool    Leeds United      -375     +510      +900  draftkings   
4   West Bromwich  Leicester City      +280     +255      -103  draftkings   

                    datetime  
0 2020-09-03 20:03:49.921738  
1 2020-09-03 20:03:49.921738  
2 2020-09-03 20:03:49.921738  
3 2020-09-03 20:03:49.921738  
4 2020-09-03 20:03:49.921738  
        home_team    away_team home_odds tie_odds away_odds   source  \
0          Fulham      Arsenal      +460     +320      -175  fanduel   
1        West Ham    Newcastle      +110     +240      +250  fanduel   
2  Crystal Palace  Southampton      +190     +220      +155  fanduel   
3       Liverpool        Leeds      -410     +550      +9

In [ ]:
Prem_Web_Scraper = Web_Scraper(prem_dir, output_cols)

In [ ]:
test = Prem_Web_Scraper.go()

In [ ]:
test.to_sql()

#### Grab Probabilities and Fixtures from 538 

In [12]:
matches_538 = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv')
prem_matches = matches_538[matches_538['league'] == 'Barclays Premier League']

columns = ['date', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2']

prem_matches[prem_matches['date']=='2020-07-16'][['proj_score1', 'proj_score2']]

,proj_score1,proj_score2
5705,1.50,1.12
5709,1.61,0.71
5723,0.65,2.00
5724,1.50,1.13


#### Connect to Remote PostgreSQL server

In [4]:
config = configparser.ConfigParser()
config.read('credentials.txt')
remote = config['remote']
DB_URI = ''.join(['postgres+psycopg2://',remote['user'],':',remote['password'],'@',remote['server'],'/',remote['database']])

conn = db.create_engine(DB_URI).connect()

In [5]:
results = pd.read_sql('SELECT * FROM test.test;', conn)
results.head()

,coltest
0,Hello World


In [11]:
conn.close()